# 子查詢抓取
1. 擷取所有Node list 
2. loop所有Node
3. 細節如下 
 - 若整體無keyword return “ ” (應該不用)
 - loop : token_next()功能抓取下一個token 判斷有沒有 "括號" 。
 - 若有keyword token ： 更新dict
 - 記錄功能列 : SELECT 、 WHERE 、FROM
 - 遇到Where 要往下伸展
 - 判斷括號進入 : subquery_trace(dict, sqlfun, tokens)
 - 迴圈結束後return dictionary

In [1]:
import sqlparse

In [2]:
the_query = '''CREATE TABLE FINAL as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)
               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) 
               Group by a1
               limit 15'''
the_query3 = '''CREATE TABLE FINAL as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    left join (SELECT * FROM MEOW.BBB_table ) as t2 on (t3.a1 = t2.a5)
               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) 
               Group by a1
               limit 15'''


In [12]:
res3 = sqlparse.parse(the_query)
stmt3 = res3[0]
stmt3.tokens

[<DDL 'CREATE' at 0x511CA08>,
 <Whitespace ' ' at 0x5142BE8>,
 <Keyword 'TABLE' at 0x5142C48>,
 <Whitespace ' ' at 0x5142CA8>,
 <Keyword 'FINAL' at 0x5142D08>,
 <Whitespace ' ' at 0x5142D68>,
 <Keyword 'as' at 0x5142DC8>,
 <Whitespace ' ' at 0x5142E28>,
 <Newline ' ' at 0x5142E88>,
 <Whitespace ' ' at 0x5142EE8>,
 <Whitespace ' ' at 0x5142F48>,
 <Whitespace ' ' at 0x5142FA8>,
 <Whitespace ' ' at 0x4E73048>,
 <Whitespace ' ' at 0x4E730A8>,
 <Whitespace ' ' at 0x4E73108>,
 <Whitespace ' ' at 0x4E73168>,
 <Whitespace ' ' at 0x4E731C8>,
 <Whitespace ' ' at 0x4E73228>,
 <Whitespace ' ' at 0x4E73288>,
 <Whitespace ' ' at 0x4E732E8>,
 <Whitespace ' ' at 0x4E73348>,
 <Whitespace ' ' at 0x4E733A8>,
 <Whitespace ' ' at 0x4E73408>,
 <Whitespace ' ' at 0x4E73468>,
 <DML 'select' at 0x4E734C8>,
 <Whitespace ' ' at 0x4E73528>,
 <IdentifierList 'a1, co...' at 0x4E67A20>,
 <Parenthesis '(a2)' at 0x4EECA98>,
 <Whitespace ' ' at 0x4E73828>,
 <Newline ' ' at 0x4E73888>,
 <Whitespace ' ' at 0x4E738E8>,
 <

## Function : 是否有子查詢

In [3]:
def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False

## Function : 追蹤token內的列表並確認是否有子查詢 

In [4]:
def trace_token_is_subselect(parsed, count_dict, state):
    subselect_dict = {} # count_dict as KEY, item as VALUE
    
    ## No group return null subquery_dict
    if not parsed.is_group:
        return subselect_dict
    
    ## With group return Record subquery_dict
    for item in parsed.tokens:
        if is_subselect(item):
            count_dict[state] += 1
            subselect_dict[ count_dict[state] ] = item
    
    return subselect_dict
    

In [142]:
the_query
subquery_dict={} # Save "query" Scripts and "tokens"
state = "MAIN" # MAIN、SELECT、FROM、WHERE
res = sqlparse.parse(the_query)
stmt = res[0]
# subquery_dict[state] = {"query" : the_query, "tokens" : stmt.tokens}

# Record subquery in differnent statement out times
count_dict = {"MAIN" : 0, "SELECT" : 0, "FROM" : 0, "WHERE" : 0}
statement = ""

## 主程式
### 1.判斷子查詢相關SQL功能
- SELECT : 判斷ttype is DML + 值為 "SELECT"
- FROM : 判斷ttype is Keyword + 值為 "FROM"
- WHERE : 判斷instance is Where + 值含 "WHERE"

### 2. 紀錄上述state作為後續標註
- state : 樣式為 # MAIN、SELECT、FROM、WHERE再持續往上加
- count_dict：紀錄不同樣式的子查詢出現次數。

In [101]:
from sqlparse.sql import IdentifierList, Identifier, Where
from sqlparse.tokens import Keyword, DML, DDL
import re
# global variables 
subquery_dict={} # Save "query" Scripts and "tokens"
state = "MAIN" # MAIN、SELECT、FROM、WHERE

# Function local variables and codes 
# Record query's tokens
res = sqlparse.parse(the_query)
stmt = res[0]
subquery_dict[state] = {"query" : the_query, "tokens" : stmt.tokens}

# Record subquery in differnent statement out times
count_dict = {"MAIN" : 0, "SELECT" : 0, "FROM" : 0, "WHERE" : 0}
statement = ""
for token in stmt.tokens:
    
    # Setting state
    if token.ttype is DML and token.value.upper() == "SELECT":
        statement = "SELECT"
        print("SELECT表示句:" + token.value.upper())
    if token.ttype is Keyword and token.value.upper() == "FROM":
        statement = "FROM"
        print("FROM表示句:" + token.value.upper())
    if isinstance(token, Where) and bool(re.search("WHERE",token.value.upper())):
        statement = "WHERE"
        print("WHERE表示句:" + token.value.upper())
        
    # is_subselect check 
    ## Record statement count and Print subselect position 
    ### 1st class trace
    ### Save subquerys and recursive
    if (is_subselect(token)):
        count_dict[statement] += 1
        the_subquery = token.value # Delete Captain and tail (parentheses)
        the_subtoken = token.tokens
        new_state = state + "_" + statement + count_dict[statement] # Subselect key set new state
        subquery_dict[new_state] = {"query" : the_subquery, "tokens" : the_subtokens}
        print(new_state)
        print(the_subquery)
    
    ### 2nd class trace    
    ### Save subquerys and recursive 
    subselect_dict = {}
    subselect_dict = trace_token_is_subselect(token, count_dict, statement) 
    for sub_item in subselect_dict.items():
        the_subquery = sub_item[1].value # Delete Captain and tail (parentheses)
        the_subtokens = sub_item[1]
        new_state = state + "_" + statement + str(sub_item[0]) # Subselect key set new state
        subquery_dict[new_state] = {"query" : the_subquery, "tokens" : the_subtokens}
        print(new_state)
        print(the_subquery)
        
    

SELECT表示句:SELECT
FROM表示句:FROM
MAIN_FROM1
(SELECT * FROM MEOW.CCC_table )


In [190]:
subquery_dict_params

NameError: name 'subquery_dict_params' is not defined

## 主程式遞迴Function

In [217]:
def split_subquery(the_query, state = "MAIN", subquery_dict_params = {}):
    # Function local variables and codes 
    # Record query's tokens
    subquery_dict = subquery_dict_params
    print(subquery_dict)
    res = sqlparse.parse(the_query)
    stmt = res[0]
    subquery_dict[state] = the_query

    # 01 Record subquery in differnent statement out times
    count_dict = {"MAIN" : 0, "SELECT" : 0, "FROM" : 0, "WHERE" : 0}
    statement = ""
    for token in stmt.tokens:

        # 02 Setting state
        if token.ttype is DML and token.value.upper() == "SELECT":
            statement = "SELECT"
            print("SELECT表示句:" + token.value.upper())
        if token.ttype is Keyword and token.value.upper() == "FROM":
            statement = "FROM"
            print("FROM表示句:" + token.value.upper())
        if isinstance(token, Where) and bool(re.search("WHERE",token.value.upper())):
            statement = "WHERE"
            print("WHERE表示句:" + token.value.upper())

        # 03 is_subselect check 
        ## Record statement count and Print subselect position 
        ### 03_1 1st class trace
        ### Save subquerys and recursive
        if (is_subselect(token)):
            count_dict[statement] += 1
            the_subquery = token.value # Delete Captain and tail (parentheses)
            the_subtoken = token.tokens
            new_state = state + "_" + statement + count_dict[statement] # Subselect key set new state
            subquery_dict[new_state] =  the_subquery
            split_subquery(the_query = the_subquery[1:-1], 
                           state = new_state, 
                           subquery_dict_params = subquery_dict)

        ### 03_2 2nd class trace    
        ### Save subquerys and recursive 
        subselect_dict = {}
        subselect_dict = trace_token_is_subselect(token, count_dict, statement) 
        for sub_item in subselect_dict.items():
            the_subquery = sub_item[1].value # Delete Captain and tail (parentheses)
            the_subtokens = sub_item[1]
            new_state = state + "_" + statement + str(sub_item[0]) # Subselect key set new state
            subquery_dict[new_state] = the_subquery
            print(new_state)
            print(the_subquery)
            ### recirsive subselect and Update the subquery_dict
            split_subquery(the_query = the_subquery[1:-1], 
                           state = new_state, 
                           subquery_dict_params = subquery_dict)
  
        
    return(subquery_dict)
    

In [134]:
count_dict

{'FROM': 1, 'MAIN': 0, 'SELECT': 0, 'WHERE': 0}

In [218]:
the_query = '''CREATE TABLE FINAL as 
               select a1, count(a2) 
               from MEOW.AAA_table as t3 
                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)
               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) 
               Group by a1
               limit 15'''
# the_query = '''SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1'''
xx = {}
xx = split_subquery(the_query, 
                    state = "MAIN",
                   subquery_dict_params={})

{}
SELECT表示句:SELECT
FROM表示句:FROM
MAIN_FROM1
(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )
{'MAIN': "CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1\n               limit 15", 'MAIN_FROM1': '(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )'}
SELECT表示句:SELECT
FROM表示句:FROM
MAIN_FROM1_FROM1
(SELECT * FROM MEOW.CCC_table )
{'MAIN': "CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1\n               limit 15", 

In [208]:
the_query2 = xx["MAIN_FROM1"][1:-1]
xx = split_subquery(the_query2, 
                    state = "MAIN_FROM1",
                   subquery_dict_params=xx)

{'MAIN': "CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1\n               limit 15", 'MAIN_FROM1': '(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )', 'MAIN_WHERE1': '(SELECT distinct(b1) FROM MEOW.BBB_table )'}
SELECT表示句:SELECT
FROM表示句:FROM
MAIN_FROM1_FROM1
(SELECT * FROM MEOW.CCC_table )
{'MAIN': "CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1\n               limit 15", 'MAIN_FROM1': 'SELECT * FROM (SELECT *

In [106]:
the_query

"CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1\n               limit 15"

In [219]:
xx

{'MAIN': "CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1\n               limit 15",
 'MAIN_FROM1': 'SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ',
 'MAIN_FROM1_FROM1': 'SELECT * FROM MEOW.CCC_table ',
 'MAIN_WHERE1': 'SELECT distinct(b1) FROM MEOW.BBB_table '}

In [59]:
xx["MAIN_FROM1"][1:-1]

'SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 '

In [60]:
split_subquery(the_query = xx["MAIN_FROM1"][1:-1], 
                           state = "MAIN_FROM1")

SELECT表示句:SELECT
FROM表示句:FROM
MAIN_FROM1_FROM1
(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )
WHERE表示句:WHERE T3.X1 = 'THING' AND T3.X2 IN (SELECT DISTINCT(B1) FROM MEOW.BBB_TABLE ) 
               
MAIN_FROM1_WHERE1
(SELECT distinct(b1) FROM MEOW.BBB_table )


{'MAIN': "CREATE TABLE FINAL as \n               select a1, count(a2) \n               from MEOW.AAA_table as t3 \n                    left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)\n               where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) \n               Group by a1\n               limit 15",
 'MAIN_FROM1': 'SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ',
 'MAIN_FROM1_FROM1': '(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )',
 'MAIN_FROM1_WHERE1': '(SELECT distinct(b1) FROM MEOW.BBB_table )',
 'MAIN_WHERE1': '(SELECT distinct(b1) FROM MEOW.BBB_table )'}

In [38]:
split_subquery(the_query = "SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1", 
                           state = "MAIN_FROM1")

SELECT表示句:SELECT
FROM表示句:FROM
MAIN_FROM1
(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )
WHERE表示句:WHERE T3.X1 = 'THING' AND T3.X2 IN (SELECT DISTINCT(B1) FROM MEOW.BBB_TABLE ) 
               
MAIN_WHERE1
(SELECT distinct(b1) FROM MEOW.BBB_table )


{'MAIN': 'SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1',
 'MAIN_FROM1': '(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )',
 'MAIN_FROM1_FROM1': '(SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 )',
 'MAIN_FROM1_WHERE1': '(SELECT distinct(b1) FROM MEOW.BBB_table )',
 'MAIN_WHERE1': '(SELECT distinct(b1) FROM MEOW.BBB_table )'}

In [36]:
subquery_dict["MAIN_FROM1"]["query"][1:-1]

'SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 '